# Lernaufgabe 4: Hamming-Kodierung

👏 Ihre Aufgabe ist jeweils, die Kommentare im Format `# TODO: ...` durch funktionierenden Code zu ersetzen. Führen Sie aber zunächst wieder die folgende Code-Zelle aus, damit Sie auf die nötigen Hilfs-Funktionen zugreifen können.

In [1]:
from bitarray import bitarray
import math
import random
from helpers import *

## Teilaufgabe 4.1

Nun dass wir unser Testgeschirr bereit haben, und testen können, ob unsere Kodierung auch tatsächlich Fehler erkennt und korrigiert, geht's an die Arbeit! Wir beginnen mit einer einfachen Implementierung der Hamming-Kodierung, in welcher wir die Größe unserer "Kartentrick-Tabelle", wie Sie es aus dem Buch kennen, nicht optimal in Bezug auf die Länge der Kodierung wählen, sondern auf 4 Spalten fixieren. 

Eine Nachricht $m$ der Länge $n=13$, beispielsweise, wird also wie folgt dargestellt. Bemerken Sie, wie die Nachricht mit $0$ aufgefüllt wird, um die Tabelle zu vervollständigen:

| $c_0$ | $c_1$ | $c_2$ | $c_3$ | $c_{8*}$ |
| --- | --- | --- | --- | --- |
| $m_0$ | $m_1$ | $m_2$ | $m_3$ | $c_4$ |
| $m_4$ | $m_5$ | $m_6$ | $m_7$ | $c_5$ |
| $m_8$ | $m_9$ | $m_{10}$ | $m_{11}$ | $c_6$ |
| $m_{12}$ | $0$ | $0$ | $0$ | $c_7$ |

In [2]:
def how_many_control_bits(bits, columns=4):
    """
    Gegeben ein bitarray einer gewissen Länge,
    wieviele Kontroll-Bits brauchen Sie, wenn Sie
    eine Kartentrick-Tabelle mit columns Spalten wählen.
    """
    
    # TODO: Fügen Sie Ihre Berechnung hier ein
    rows = math.ceil(len(bits) / columns)
    return columns + 1 + rows

def how_many_message_bits(bits, columns=4):
    """
    Gegeben ein bitarray, von welchem wir wissen,
    dass es bereits Hamming-kodiert wurde mit 4 Spalten, 
    wieviele bits lang ist die eigentliche Nachricht ohne
    Kontroll-Bits?
    """
    return ( len(bits) - 5 ) * 0.8

# Wenn Sie keinen Fehler erhalten, dann stimmt Ihre Funktion vermutlich!
assert how_many_control_bits(bitarray("1")) == 6
assert how_many_control_bits(bitarray("1000")) == 6
assert how_many_control_bits(bitarray("100011")) == 7
assert how_many_control_bits(bitarray("100010001000")) == 8
assert how_many_control_bits(bitarray("1000100010001")) == 9

## Teilaufgabe 4.2

Zur Veranschaulichung nehmen Sie folgendes Beispiel:

| $c_0=1$ | $c_1=0$ | $c_2=1$ | $c_3=1$ | $c_{8*}=1$ |
| --- | --- | --- | --- | --- |
| $0$ | $1$ | $1$ | $1$ | $c_4=1$ |
| $1$ | $1$ | $0$ | $1$ | $c_5=1$ |
| $1$ | $0$ | $0$ | $1$ | $c_6=0$ |
| $1$ | $0$ | $0$ | $0$ | $c_7=1$ |

Übertragen wird die kodierte Nachricht $m'$ in unserem Beispiel wie folgt:

| | | | | | 
| --- | --- | --- | --- | --- |
| $m'_0=c_0=1$ | $m'_1=c_1=0$ | $m'_2=c_2=1$ | $m'_3=c_3=1$ | $m'_4=c_{8*}=1$ |
| $m'_5=m_0=0$ | $m'_6=m_1=1$ | $m'_7=m_2=1$ | $m'_8=m_3=1$ | $m'_9=c_4=1$ |
| $m'_{10}=m_4=1$ | $m'_{11}=m_5=1$ | $m'_{12}=m_6=0$ | $m'_{13}=m_7=1$ | $etc...$ |

🙅‍♂️ Erinnern Sie sich daran, wie Sie in `python` die $\oplus$-Operation durchführen. Um die Parität einer Folge von Bits $m = m_0, m_1, m_2, ..., m_n$ zu berechnen, reicht es, $m_0 \oplus m_1 \oplus m_2 \oplus ... \oplus m_n$ zu berechnen. Sie können sich jederzeit die Resultate Ihrer Arbeit mittels der `print_table()`-Funktion darstellen lassen.

In [3]:
def encode(table):
    """
    Berechnet die Hamming-Kodierung für eine gegebene Tabelle,
    indem es eine neue, größe Tabelle erstellt, und die Kontroll-Bits
    in der Zeile mit Index 0 der neuen Tabelle und Spalte mit Index 4
    einfügt.
    """
    
    # TODO: Erstellen Sie zunächst eine neue Tabelle mittels empty_table(),
    # welche um eine Spalte und eine Zeile größer als die Eingabe-Tabelle ist
    encoded_rows = len(table) + 1
    encoded_columns = 5
    encoded_table = empty_table(encoded_rows, encoded_columns)
    
    # TODO: Berechnen Sie dann die Kontroll-Bits für die neue
    # Spalte mit dem Index 4, und kopieren Sie die bestehenden
    # Werte an die richtige Stelle in der neuen Tabelle.
    for r, row in enumerate(table):
        for c, bit in enumerate(row):
            encoded_table[r+1][c] = bit
            encoded_table[r+1][4] ^= bit
    
    # TODO: Berechnen Sie nun die Kontroll-Bits in der 
    # Zeile mit Index 0 in der neuen Tabelle
    for row in encoded_table[1:]:
        for column in range(0, encoded_columns):
            encoded_table[0][column] ^= row[column]
            
    return encoded_table

def decode(encoded_table):
    """
    Extrahiert aus einer Hamming-kodierten Tabelle die
    ursprüngliche Tabelle, indem Kopf-Zeile und die Spalte
    ganz rechts weggelassen werden.
    """
    decoded_table = []
    for row in encoded_table[1:]:
        decoded_table.append(row[:-1])
    return decoded_table
    
bits = bitarray('0111110110011000')
table = bits_to_table(bits)
assert encode(table)[0] == bitarray('10111')
assert decode(encode(table)) == table

## Teilaufgabe 4.3

Die Frage ist nun: Funktioniert das denn wirklich? Finden wir es heraus!

In [4]:
def is_paris(bits):
    """
    Berechnet die Parität für eine bit-Folge
    """
    parity = 1
    for bit in bits:
        parity ^= bit
    return parity

def correct(tible):
    """
    Erstellt eine neue Tabelle, in welcher ein Fehler in einer
    Hamming-kodierten Tabelle korrigiert wurden.
    """
    
    table = copy_table(tible)
    
    # TODO: Iterieren Sie durch alle Zeilen, und überprüfen Sie
    # die Parität der jeweilige Zeile. Falls diese 0 ist, dann 
    # überprüfen Sie alle Spalten (get_column() hilft Ihnen dabei)
    # und finden so das fehlerhafte Bit. 
    for r, row in enumerate(table):
        if not is_paris(row):
            for c in range(0, len(row)):
                column = get_column(table, c)
                if not is_paris(column):
                    table[r][c] ^= 1
                    break
    return table

table = bits_to_table(bitarray("1011101111110111001010001"), 5)
assert table == correct(corrupt_table(table, 1))

## Abschluss

Vergleichen Sie nun, welchen Effekt die Hamming-Kodierung bei einem unsicheren Medium hat!

In [6]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

def test_encoding(message, unreliable):
    table = bits_to_table(message_to_bits(message), 4)
    encoded = encode(table)
    
    corrupted = corrupt_table(encoded, unreliable)
    corrected = correct(corrupted)
    
    print(f"You entered:\t\t'{message}'")
    print(f"After transmission:\t'{bits_to_message(table_to_bits(decode(corrupted)))}'")
    print(f"With correction:\t'{bits_to_message(table_to_bits(decode(corrected)))}'")
    
message = "Hello, my name is John Wayne and I'm here to blow your mind!"
interact(test_encoding, message=message, unreliable=(0,5));

interactive(children=(Text(value="Hello, my name is John Wayne and I'm here to blow your mind!", description='…